In [1]:
TRAIN_PATH = ''
TEST_PATH = ''
labels_=[]

import library

In [2]:
import mediapipe as mp
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import warnings
warnings.filterwarnings('ignore')

In [3]:
df = pd.read_csv("./train.csv")
df.loc[df["label"] == labels_[0], "label"] = 0
df.loc[df["label"] == labels_[1], "label"] = 1
df.loc[df["label"] == labels_[2], "label"] = 2

,label,nose_x,nose_y,nose_z,nose_v,left_shoulder_x,left_shoulder_y,left_shoulder_z,left_shoulder_v,right_shoulder_x,...,right_heel_z,right_heel_v,left_foot_index_x,left_foot_index_y,left_foot_index_z,left_foot_index_v,right_foot_index_x,right_foot_index_y,right_foot_index_z,right_foot_index_v
28517,1,0.735630,0.543294,0.007467,0.999246,0.695831,0.417349,0.155194,0.995723,0.720067,...,0.086010,0.966131,0.226601,0.598075,0.219305,0.470830,0.220079,0.614120,0.026265,0.934942
28518,1,0.775572,0.517579,0.012821,0.999378,0.704168,0.404210,0.162908,0.995909,0.730823,...,0.070911,0.967070,0.238810,0.610591,0.198591,0.496140,0.228907,0.625559,0.018591,0.938905
28519,1,0.790600,0.498958,0.007789,0.999467,0.710651,0.394019,0.164441,0.996123,0.736771,...,0.085872,0.967943,0.238197,0.609329,0.233198,0.510583,0.227823,0.626068,0.036127,0.940917


In [4]:
X = df.drop("label", axis=1)
y = df["label"].astype("int")

sc = StandardScaler()
sc.fit(X)
X = pd.DataFrame(sc.transform(X))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
with open("./models/plank_input_scaler.pkl", "wb") as f:
    pickle.dump(sc, f)

train ai

In [6]:
from sklearn.ensemble import RandomForestClassifier

In [7]:
rf = RandomForestClassifier(n_estimators=120,
                          max_depth=50,
                          min_samples_split=5,
                          min_samples_leaf=5,
                          max_features=None,
                          oob_score=True,
                          random_state=42)
rf.fit(X_train, y_train)


RandomForestClassifier(max_depth=50, max_features=None, min_samples_leaf=5,
                       min_samples_split=5, n_estimators=120, oob_score=True,
                       random_state=42)

In [8]:
test_df = pd.read_csv("./test.csv")
test_df = test_df.sample(frac=1).reset_index(drop=True)

test_df.loc[test_df["label"] == labels_[0], "label"] = 0
test_df.loc[test_df["label"] == labels_[1], "label"] = 1
test_df.loc[test_df["label"] == labels_[2], "label"] = 2

test_x = test_df.drop("label", axis=1)
test_y = test_df["label"].astype("int")

test_x = pd.DataFrame(sc.transform(test_x))

In [9]:
model_results = rf.predict(test_x)

In [10]:
from sklearn.metrics import precision_score, accuracy_score, f1_score, recall_score, confusion_matrix

In [11]:
p_score = precision_score(test_y, model_results, average=None, labels=[0, 1, 2])
a_score = accuracy_score(test_y, model_results)
r_score = recall_score(test_y, model_results, average=None, labels=[0, 1, 2])
f1_score_result = f1_score(test_y, model_results, average=None, labels=[0, 1, 2])
cm = confusion_matrix(test_y, model_results, labels=[0, 1, 2])

In [12]:
print('RandomForest')
print(f'P_Score is {p_score}')
print(f'Accuracy_Score is {a_score}')
print(f'Recall_Score is {r_score}')
print(f'F1_Score is {f1_score_result}')

RandomForest
P_Score is [0.74038462 1.         0.98125   ]
Accuracy_Score is 0.8816901408450705
Recall_Score is [0.98717949 0.98755187 0.66808511]
F1_Score is [0.84615385 0.99373695 0.79493671]


In [13]:
with open("./models/plank.pkl", "wb") as f:
    pickle.dump(rf, f)